# Misinformation Detection
### Dataset Loading and Combination Notebook

*Session 20 Group 4*
*Erica, Sahan, Dinuka*


Imports and setup

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
from bs4 import BeautifulSoup
import re
import unidecode
from bs4 import MarkupResemblesLocatorWarning
import warnings

warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

### Load datasets

In [2]:
provided_dataset_paths = ["data/provided/Constraint_English_Test.xlsx", "data/provided/Constraint_English_Train.xlsx", "data/provided/Constraint_English_Val.xlsx"]
dataset_1_path = "data/fake-news-detection_10k/fake_and_real_news.csv"
dataset_2_path = "data/fake-real-news_10k/dataset.csv"
dataset_3_path_real = "data/misinformation-fake-news-text-dataset_79k/DataSet_Misinfo_TRUE.csv"
dataset_3_path_fake = "data/misinformation-fake-news-text-dataset_79k/DataSet_Misinfo_FAKE.csv"

In [6]:
dataset_0 = pd.concat((pd.read_excel(f, index_col=0) for f in provided_dataset_paths), ignore_index=True)

print("Provided dataset (0):\n"+str(dataset_0.count().iloc[0]))
display(dataset_0.head())

dataset_1 = pd.read_csv(dataset_1_path)

print("Dataset 1:\n"+str(dataset_1.count().iloc[0]))
display(dataset_1.head())

dataset_2 = pd.read_csv(dataset_2_path, encoding='latin-1')

print("Dataset 2:\n"+str(dataset_2.count().iloc[0]))
display(dataset_2.head())

dataset_3_real = pd.read_csv(dataset_3_path_real, index_col=0)

print("Dataset 3 (Real):\n"+str(dataset_3_real.count().iloc[0]))
display(dataset_3_real.head())

dataset_3_fake = pd.read_csv(dataset_3_path_fake, index_col=0)

print("Dataset 3 (Fake):\n"+str(dataset_3_fake.count().iloc[0]))
display(dataset_3_fake.head())

Provided dataset (0):
10700


,tweet,label
0,Our daily update is published. States reported...,real
1,Alfalfa is the only cure for COVID-19.,fake
2,President Trump Asked What He Would Do If He W...,fake
3,States reported 630 deaths. We are still seein...,real
4,This is the sixth time a global health emergen...,real


Dataset 1:
9900


,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


Dataset 2:
9960


,News_Headline,Link_Of_News,Source,Stated_On,Date,Label
0,Says Osama bin Laden endorsed Joe Biden,https://www.politifact.com/factchecks/2020/jun...,Donald Trump Jr.,"June 18, 2020","June 19, 2020",FALSE
1,CNN aired a video of a toddler running away fr...,https://www.politifact.com/factchecks/2020/jun...,Donald Trump,"June 18, 2020","June 19, 2020",pants-fire
2,Says Tim Tebow kneeled in protest of abortion...,https://www.politifact.com/factchecks/2020/jun...,Facebook posts,"June 12, 2020","June 19, 2020",FALSE
3,Even so-called moderate Democrats like Joe Bi...,https://www.politifact.com/factchecks/2020/jun...,Paul Junge,"June 10, 2020","June 19, 2020",barely-true
4,"""Our health department, our city and our count...",https://www.politifact.com/factchecks/2020/jun...,Jeanette Kowalik,"June 14, 2020","June 18, 2020",TRUE


Dataset 3 (Real):
34946


,text
0,The head of a conservative Republican faction ...
1,Transgender people will be allowed for the fir...
2,The special counsel investigation of links bet...
3,Trump campaign adviser George Papadopoulos tol...
4,President Donald Trump called on the U.S. Post...


Dataset 3 (Fake):
43642


,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...


### Things to fix so that datasets can be combined:

Dataset 0:
- Rename tweet column to 'text'

Dataset 1:
- Rename 'Text' column to 'text'
- Set labels to lowercase

Dataset 2:
- Drop extra columns
- Consider keeping as a separate dataset with the extra columns for an alternative model
- Make labels properly binary. Currently: (TRUE, mostly-true, half-true, barely-true, FALSE, pants-fire). Set so that >= half-true is real, and anything else is fake.

Dataset 3:
- Combine into one dataframe with a label column

Once all of the above is done, the datasets can be combined and further preprocessing can be done to ready the data for visualisation or training.

Dataset 0:

In [ ]:
dataset_0 = dataset_0.rename(columns={"tweet": "text", "label": "label"}) # Rename columns
dataset_0.head()

,text,label
0,Our daily update is published. States reported...,real
1,Alfalfa is the only cure for COVID-19.,fake
2,President Trump Asked What He Would Do If He W...,fake
3,States reported 630 deaths. We are still seein...,real
4,This is the sixth time a global health emergen...,real


Dataset 1:

In [ ]:
dataset_1.columns = map(str.lower, dataset_1.columns) # Make labels lowercase
dataset_1["label"] = dataset_1["label"].str.lower() # Lowercase column names

dataset_1.head()

,text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,fake
1,U.S. conservative leader optimistic of common ...,real
2,"Trump proposes U.S. tax overhaul, stirs concer...",real
3,Court Forces Ohio To Allow Millions Of Illega...,fake
4,Democrats say Trump agrees to work on immigrat...,real


Dataset 2:

In [ ]:
dataset_2 = dataset_2.drop(labels=["Link_Of_News", "Source", "Stated_On", "Date"], axis=1) # Drop unneeded additional columns
dataset_2 = dataset_2.rename(columns={"News_Headline": "text", "Label": "label"}) # Rename remaining columns

# Function to turn variable labels into just "real" or "fake"
def binarise_label(label_text):
    if label_text in ["TRUE", "mostly-true", "half-true"]:
        return "real"
    elif label_text in ["barely-true", "FALSE", "pants-fire"]:
        return "fake"
    else:
        return ""
    
dataset_2['label'] = dataset_2['label'].map(binarise_label)

dataset_2.head()

,text,label
0,Says Osama bin Laden endorsed Joe Biden,fake
1,CNN aired a video of a toddler running away fr...,fake
2,Says Tim Tebow kneeled in protest of abortion...,fake
3,Even so-called moderate Democrats like Joe Bi...,fake
4,"""Our health department, our city and our count...",real


Dataset 3:

In [ ]:
# Create label columns for single-label sub-datasets
dataset_3_fake['label'] = "fake"
dataset_3_real['label'] = "real"

dataset_3 = pd.concat([dataset_3_fake, dataset_3_real]) # Concatenate sub-datasets
dataset_3 = dataset_3.sample(frac=1) # Randomise the row order
dataset_3 = dataset_3.reset_index(drop=True)
dataset_3.head()

,text,label
0,Hillary Clinton proposed a drone attack on the...,fake
1,Political activist and Hillary supporter Amal ...,fake
2,Just when you thought Republicans couldn t be ...,fake
3,"CNN, the news division of Time Warner Inc, sai...",real
4,The insanity continues! The outrageous invasio...,fake


Combine all datasets into one:

In [8]:
main_dataset = pd.concat([dataset_0, dataset_1, dataset_2, dataset_3])
main_dataset = main_dataset.sample(frac=1)
main_dataset = main_dataset.reset_index(drop=True)

display(main_dataset.count())
main_dataset.head()

text     109148
label    109177
dtype: int64

,text,label
0,The Reason Why Jade Helm Is Obama’s Favorite C...,fake
1,U.S. President Donald Trump s revised hardline...,real
2,A Brazilian congressional report recommended o...,real
3,"Ah, The Simpsons, the hilarious animated show ...",fake
4,Says former FBI Director James Comey admitted ...,fake


Save into a file:

In [9]:
main_dataset.to_csv("data/total_dataset.csv")

Load from file:

In [39]:
main_dataset = pd.read_csv("data/total_dataset.csv", index_col=0)
main_dataset.head()

,text,label
0,"The Reason Why Jade Helm Is Obama’s Favorite Conspiracy Theory May Surprise You By Kristie on November 22, 2015 Subscribe \nPresident Obama has had to deal with his fair share of conspiracy theories. Even before he was elected he had to deal with questions about his origin of birth. He’s also secretly a Muslim masquerading as a Christian. One of the best is that he is going to unarm Americans by taking our guns. I’m still waiting on that one. But in a recent GQ interview , President Obama revealed his favorite conspiracy theory – Jade Helm. Featured image by Boricuaeddie , available under a Creative Commons Attribution-Noncommercial license. \nAs most will recall, Jade Helm was one of the largest military training exercise ever conducted on U.S. soil. In July of this year, roughly 1,200 U.S. military troops participated in exercises that spanned seven states. Jade Helm included New Mexico, Arizona, and Texas. As we all know though, this exercise proved itself to be a great opportunity for the bitter right-wing to start up the conspiracy machine. \nWhile this resulted in great ratings for the likes of Alex Jones , Jade Helm turned out to be a — military exercise — not a military led plot for Obama to take over the country and stay in power as a dictator. No wonder Jade Helm entertained Obama. Only a right-wing nut-job would believe something like that. \nBut why is that Obama’s favorite conspiracy theory? He explained after GQ asked : “What’s the most entertaining conspiracy theory you ever read about yourself?” \nObama then answered : “That military exercises we were doing in Texas were designed to begin martial law so that I could usurp the Constitution and stay in power longer. Anybody who thinks I could get away with telling Michelle I’m going to be president any longer than eight years does not know my wife.” \nYou see, even the leader of the free world answers to somebody. About Kristie \nKristie is 22-years-old and resides in Nashville, TN. While reading is a passion, she also has a passion for writing. Reporting on social issues such as LGBT rights, racial injustices, and religious intolerance, she also has a vested interest in the current political climate in America. Connect",fake
1,"U.S. President Donald Trump s revised hardline policy towards Iran will have little impact on Tehran s ambition to develop its vital oil industry and attract foreign investment, a senior Iranian official said on Tuesday. The Islamic Republic hopes to signs 10 new deals with foreign companies to develop oil and gas fields by March 2018, Amir Zamaninia, Iran s deputy oil minister for trade and international affairs, told the Oil & Money conference in London. Trump s decision last Friday not to certify that Iran was complying with last year s international deal on its nuclear power development program will not detract from Tehran s plans, he said. I don t think any of us, or any of our partners have been surprised by this statement. Our expectations of policy formulation from Washington have been very measured, he said. The (U.S.) statement and policy ... has little or no effect and implication on our future plan in the oil industry, Zamaninia said. Iran is negotiating 28 contracts with foreign companies, including many of Europe s top oil companies, under a new development contract, he added. France s Total (TOTF.PA) became in July the first Western oil major to re-enter Iran following the lifting of international sanctions with an agreement to develop the giant South Pars offshore gas field. Total s chief executive officer said in an interview with International Oil Daily the company will wait to see the impact of Trump s decision and that it will try to continue with it if the United States imposes unilateral sanctions. We have signed a contract in Iran. If we can move forward, we ll move forward. If we cannot, we will have to stop, Patrick Pouyanne said. Royal Dutch Shell (RDSa.L) has also signed an agreement with Iran for

### Cleaning dataset

Remove blank rows:

In [40]:
dataset = main_dataset
print(f"Before culling:\n {dataset.count()}")
dataset = dataset.dropna()
dataset = dataset.astype(str).replace(r'^\s*$', np.nan, regex=True)
dataset = dataset.dropna()
print(f"After culling:\n {dataset.count()}")

Before culling:
 text     109148
label    109072
dtype: int64
After culling:
 text     109043
label    109043
dtype: int64


Feature extraction by bag-of-words, TF-IDF, or semantic embeddings benefits from consistent and simplified text without punctuation, capitalisation, empty spaces, formatting, or other non-meaningful words like links or isolated characters.

In [ ]:
# Remove newlines and tabs
def remove_newlines_tabs(text):
    formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com') # Also join the . and com together in links for consistency.
    return formatted_text

# Convert typographical apostrophes/quotemarks to regular apostrophes
def normalise_apostrophes(text):
    normalised_text = text.replace("’", "'").replace("‘", "'")
    return normalised_text

# Strip any html tags
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

# Remove accented characters
def accented_characters_removal(text):
    decoded_text = unidecode.unidecode(text)
    return decoded_text

# Strip links from the text
def remove_links(text):
    remove_https = re.sub(r'http\S+', '', text)
    remove_com = re.sub(r'\ [A-Za-z]*\.com', ' ', remove_https)
    return remove_com

# Make everything lowercase for consistency between characters and semantics
def set_lowercase(text):
    lower_text = text.lower()
    return lower_text

# Reduce repeated characters and punctuation
# This will cut down long strings of the same letter repeated to just two instances.
# e.g. Amaaaaaaaaaaazzzzzinggg!!!!!! -> Amaazzingg!!!!!!
def reduce_character_repetition(text):
    pattern_alpha = re.compile(r'([A-Za-z])\1{2,}', re.DOTALL) # Find all instances of repetition
    formatted_text = pattern_alpha.sub(r'\1\1', text) # Limit repetition to two characters for each instance
    return formatted_text

CONTRACTIONS_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
}

# Expand contractions into multiple words
def expand_contractions(text, contraction_map=CONTRACTIONS_MAP):
    word_list = text.split(' ')
    for word in contraction_map.keys():
        if word in word_list:
            word_list = [item.replace(word, contraction_map[word]) for item in word_list]
            
    new_string = ' '.join(str(expanded) for expanded in word_list)
    return new_string

# Remove the first period from the word "U.S." specifically without adding spaces
def fix_usa(text):
    fixed_text = text.replace("u.s", "us")
    return fixed_text

# Remove special characters
def remove_special_characters(text):
    processed_text = re.sub(r'\W', ' ', text)
    processed_text = processed_text.replace('_', ' ') # Underscores are counted as a word character, but we remove them as well.
    return processed_text

# Remove isolated letters 
def removed_isolated_letters(text):
    stripped_text = re.sub(r'(?<=\s)[a-zA-Z](?=\s)', ' ', text) # Middle
    stripped_text = re.sub(r'^[a-zA-Z](?=\s)', '', stripped_text) # Start of string
    stripped_text = re.sub(r'(?<=\s)[a-zA-Z]$', '', stripped_text) # End of string
    return stripped_text

# Condense whitespace
def condense_whitespace(text):
    without_whitespace = re.sub(r'\s+', ' ', text)
    without_whitespace = without_whitespace.strip() # Remove from start and end of string.
    return without_whitespace

dataset['text'] = dataset['text'].map(remove_newlines_tabs) # Removes newlines and tabs
print("Newlines and tabs removed.")
dataset['text'] = dataset['text'].map(normalise_apostrophes) # Standardises typographical quotes and apostrophes into ' and "
print("Apostrophes normalised")
dataset['text'] = dataset['text'].map(strip_html_tags) # Removes HTML tags
print("HTML tags removed.")
dataset['text'] = dataset['text'].map(accented_characters_removal) # Simplifies accented characters into their english equivalents
print("Accented characters removed.")
dataset['text'] = dataset['text'].map(remove_links) # Finds and removes links.
print("Links removed.")
dataset['text'] = dataset['text'].map(set_lowercase) # Makes all text lowercase
print("Text made lowercase.")
dataset['text'] = dataset['text'].map(reduce_character_repetition) # Reduces repetition of the same letter in a row to max of 2.
print("Reduced character repetition.")
dataset['text'] = dataset['text'].map(lambda x: expand_contractions(x)) # Expands contractions. you're -> you are
print("Contractions expanded.")
dataset['text'] = dataset['text'].map(fix_usa) # Removes the period in U.S. to just be US
print("U.S. formatting normalised.")
dataset['text'] = dataset['text'].map(remove_special_characters) # Removes all punctuation and special characters.
print("Special characters removed.")
dataset['text'] = dataset['text'].map(removed_isolated_letters) # Removes single letters sitting on their own.
print("Isolated letters removed.")
dataset['text'] = dataset['text'].map(condense_whitespace) # Condenses multiple spaces into one space.
print("Whitespace condensed.")

# Final dropping of empty rows after all processing
dataset = dataset.dropna()
dataset = dataset.astype(str).replace(r'^\s*$', np.nan, regex=True)
dataset = dataset.dropna()
dataset = dataset.reset_index(drop=True)
print("Empty rows dropped.")

print("\nAll processing completed.")

pd.set_option("display.max_colwidth", None)
dataset.head()


Newlines and tabs removed.
Apostrophes normalised
HTML tags removed.
Accented characters removed.
Links removed.
Text made lowercase.
Reduced character repetition.
Contractions expanded.
U.S. formatting normalised.
Special characters removed.
Isolated letters removed.
Whitespace condensed.
Empty rows dropped.

All processing completed.


,text,label
0,the reason why jade helm is obama favorite conspiracy theory may surprise you by kristie on november 22 2015 subscribe president obama has had to deal with his fair share of conspiracy theories even before he was elected he had to deal with questions about his origin of birth he is also secretly muslim masquerading as christian one of the best is that he is going to unarm americans by taking our guns am still waiting on that one but in recent gq interview president obama revealed his favorite conspiracy theory jade helm featured image by boricuaeddie available under creative commons attribution noncommercial license as most will recall jade helm was one of the largest military training exercise ever conducted on us soil in july of this year roughly 1 200 us military troops participated in exercises that spanned seven states jade helm included new mexico arizona and texas as we all know though this exercise proved itself to be great opportunity for the bitter right wing to start up the conspiracy machine while this resulted in great ratings for the likes of alex jones jade helm turned out to be military exercise not military led plot for obama to take over the country and stay in power as dictator no wonder jade helm entertained obama only right wing nut job would believe something like that but why is that obama favorite conspiracy theory he explained after gq asked what the most entertaining conspiracy theory you ever read about yourself obama then answered that military exercises we were doing in texas were designed to begin martial law so that could usurp the constitution and stay in power longer anybody who thinks could get away with telling michelle am going to be president any longer than eight years does not know my wife you see even the leader of the free world answers to somebody about kristie kristie is 22 years old and resides in nashville tn while reading is passion she also has passion for writing reporting on social issues such as lgbt rights racial injustices and religious intolerance she also has vested interest in the current political climate in america connect,fake
1,us president donald trump revised hardline policy towards iran will have little impact on tehran ambition to develop its vital oil industry and attract foreign investment senior iranian official said on tuesday the islamic republic hopes to signs 10 new deals with foreign companies to develop oil and gas fields by march 2018 amir zamaninia iran deputy oil minister for trade and international affairs told the oil money conference in london trump decision last friday not to certify that iran was complying with last year international deal on its nuclear power development program will not detract from tehran plans he said don think any of us or any of our partners have been surprised by this statement our expectations of policy formulation from washington have been very measured he said the us statement and policy has little or no effect and implication on our future plan in the oil industry zamaninia said iran is negotiating 28 contracts with foreign companies including many of europe top oil companies under new development contract he added france total totf pa became in july the first western oil major to re enter iran following the lifting of international sanctions with an agreement to develop the giant south pars offshore gas field total chief executive officer said in an interview with international oil daily the company will wait to see the impact of trump decision and that it will try to continue with it if the united states imposes unilateral sanctions we have signed contract in iran if we can move forward we ll move forward if we cannot we will have to stop patrick pouyanne said royal dutch shell rdsa has also signed an agreement with iran for the possible development of oil and gas fields iran which holds the world largest gas reserves plans to boost its gas production to 1 1 billion cubic meters per day by 2021 from the c

Save processed dataset for use with training and analysis;

In [ ]:
dataset.to_csv("data/processed_dataset.csv")